In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pytz import timezone
from datetime import datetime
from itertools import zip_longest
from skyfield import almanac, earthlib
from skyfield.api import Loader, Topos

In [2]:
load = Loader('./data')

In [3]:
ts = load.timescale()

[#################################] 100% deltat.data
[#################################] 100% deltat.preds
[#################################] 100% Leap_Second.dat


In [4]:
ephemeris = load('de421.bsp')

sun = ephemeris['sun']
earth = ephemeris['earth']

[#################################] 100% de421.bsp


In [5]:
stations_url = 'http://celestrak.com/NORAD/elements/stations.txt'
satellites = load.tle(stations_url, reload=True)

iss = satellites['ISS']

[#################################] 100% stations.txt


In [6]:
manhattan_beach_ca_usa = Topos(latitude='33.881519 N', longitude='118.388177 W', elevation_m=33)
pacific = timezone('US/Pacific')

In [7]:
# from https://docs.python.org/3/library/itertools.html, "Itertools Recipes"

def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [8]:
cardinal_points = [ pair for pair in zip([ (22.5 * i) + 11.25 for i in range(16) ], 
                                         ['N', 'NNE', 'NE', 'ENE', 
                                          'E', 'ESE', 'SE', 'SSE', 
                                          'S', 'SSW', 'SW', 'WSW', 
                                          'W', 'WNW', 'NW', 'NNW']) ]

def direction(az):
    for pt in cardinal_points:
        if az < pt[0]:
            return pt[1]
    return 'N'

In [9]:
# based on method from https://www.celestrak.com/columns/v03n01/, "Visually Observing Earth Satellites"

def eclipse_condition(satellite, earth, sun, t):
    R_e = earthlib.earth_radius_au
    R_s = 0.00465 # Sun's average radius in AU
    sat = earth + satellite
    barycentric_e = sat.at(t).observe(earth)
    barycentric_s = sat.at(t).observe(sun)
    _, _, rho_e = barycentric_e.radec()
    _, _, rho_s = barycentric_s.radec()
    theta_e = np.arcsin(R_e/rho_e.au)
    theta_s = np.arcsin(R_s/rho_s.au)
    theta = barycentric_e.separation_from(barycentric_s).radians
    umbral = np.logical_and(theta_e > theta_s, theta < (theta_e - theta_s))
    penumbral = np.logical_and(np.abs(theta_e - theta_s) < theta, theta < (theta_e + theta_s))
    annular = np.logical_and(theta_s > theta_e, theta < (theta_s - theta_e))
    return umbral, penumbral, annular

In [10]:
def nautical_twilight(topos, earth, sun, t):
    location = earth + topos
    astrocentric = location.at(t).observe(sun).apparent()
    alt, _, _ = astrocentric.altaz('standard')
    return alt.degrees < -6.

In [11]:
def above_n_degrees_alt(topos, satellite, n, t):
    difference = satellite - topos
    topocentric = difference.at(t)
    alt, _, _ = topocentric.altaz('standard')
    return alt.degrees > n 

In [12]:
# need to figure out how to make this use observer location rather than earth center

def phase_angle(satellite, earth, sun, t):
    sat = earth + satellite
    barycentric_e = sat.at(t).observe(earth)
    barycentric_s = sat.at(t).observe(sun)
    return barycentric_e.separation_from(barycentric_s).radians

In [13]:
# based on method from 
# https://astronomy.stackexchange.com/questions/28744/calculating-the-apparent-magnitude-of-a-satellite,
# "Calculating the apparent magnitude of a satellite"

std_intrinsic_mag = -1.3

def apparent_magnitude(intrinsic_magnitude, distances, phase_angles):
    term_2 = +5.0 * np.log10(distances/1000.)
    arg = np.sin(phase_angles) + (np.pi - phase_angles) * np.cos(phase_angles)
    term_3 = -2.5 * np.log10(arg)
    return intrinsic_magnitude + term_2 + term_3

In [14]:
# based on definition of sunrise_sunset in skyfields.almanac

def satellite_pass(topos, satellite, earth, sun):
    difference = satellite - topos    
    def sat_is_above_horizon(t):
        return above_n_degrees_alt(topos, satellite, 10., t)
    sat_is_above_horizon.rough_period = 0.01
    return sat_is_above_horizon

In [15]:
# based on definition of sunrise_sunset in skyfields.almanac

def visible_satellite_pass(topos, satellite, earth, sun):
    difference = satellite - topos    
    def sat_is_visible(t):
        umbral_eclipse, _, _ = eclipse_condition(satellite, earth, sun, t)
        return np.logical_and(np.logical_and(above_n_degrees_alt(topos, satellite, 10., t), 
                                             nautical_twilight(topos, earth, sun, t)), 
                              np.logical_not(umbral_eclipse))
    sat_is_visible.rough_period = 0.01
    return sat_is_visible

In [16]:
def pass_datetimes(start, end, intervals=60):
    start_datetime = start.utc_datetime()
    end_datetime = end.utc_datetime()
    delta = (end_datetime - start_datetime) / intervals
    return [ start_datetime + delta*i for i in range(intervals) ]

In [17]:
def satellite_alt(topos, satellite, t):
    difference = satellite - topos    
    topocentric = difference.at(t)
    alt, _, _ = topocentric.altaz('standard')
    return alt.degrees

In [18]:
def visible_passes(dt0, dt1, topos, satellite, earth, sun):    
    t, y = almanac.find_discrete(ts.utc(dt0), ts.utc(dt1), visible_satellite_pass(manhattan_beach_ca_usa, iss, earth, sun))
    passes = []
    difference = satellite - topos
    for pass_times in grouper(t, 2):
        start = pass_times[0]
        end = pass_times[1]
        t_p = ts.utc(pass_datetimes(start, end))
        y_p = satellite_alt(manhattan_beach_ca_usa, iss, t_p)
        culmination_alt = np.amax(y_p)
        culmination = t_p[np.where(y_p == np.amax(y_p))[0][0]]
        topocentric = difference.at(start)
        alt_start, az_start, d_start = topocentric.altaz('standard')
        topocentric = difference.at(culmination)
        alt_culm, az_culm, d_culm = topocentric.altaz('standard')
        topocentric = difference.at(end)
        alt_end, az_end, d_end = topocentric.altaz('standard')
        passes.append({ "date": start.astimezone(timezone('US/Pacific')).isoformat(' ', timespec='seconds')[:11],
                          "brightness": np.round(apparent_magnitude(std_intrinsic_mag, 
                                                                d_culm.km, 
                                                                phase_angle(iss, earth, sun, culmination)), 
                                             1),
                          "start": start.astimezone(timezone('US/Pacific')).isoformat(' ', timespec='seconds')[11:19], 
                          "start_alt": np.floor(alt_start.degrees), 
                          "start_az": direction(az_start.degrees),
                          "start_d": np.floor(d_start.km),
                          "culmination": culmination.astimezone(timezone('US/Pacific')).isoformat(' ', 
                                                                                              timespec='seconds')[11:19], 
                          "culm_alt": np.floor(alt_culm.degrees), 
                          "culm_az": direction(az_culm.degrees),
                          "culm_d": np.floor(d_culm.km),
                          "end": end.astimezone(timezone('US/Pacific')).isoformat(' ', timespec='seconds')[11:19], 
                          "end_alt": np.floor(alt_end.degrees), 
                          "end_az": direction(az_end.degrees),
                          "end_d": np.floor(d_end.km)
                         })
    return pd.DataFrame(passes)[["date", "brightness", "start", "start_alt", "start_az", "start_d",
                                     "culmination", "culm_alt", "culm_az", "culm_d",
                                     "end", "end_alt", "end_az", "end_d"]]

In [19]:
dt0 = pacific.localize(datetime(2019, 6, 1, 0, 0))
dt1 = pacific.localize(datetime(2019, 6, 11, 0, 0))
df = visible_passes(dt0, dt1, manhattan_beach_ca_usa, iss, earth, sun)

In [20]:
df

,date,brightness,start,start_alt,start_az,start_d,culmination,culm_alt,culm_az,culm_d,end,end_alt,end_az,end_d
0,2019-06-01,-2.0,22:51:29,10.0,NW,1496.0,22:52:56,23.0,NW,930.0,22:52:58,23.0,NW,921.0
1,2019-06-02,-2.6,22:02:13,10.0,NNW,1495.0,22:05:14,33.0,NE,706.0,22:05:17,33.0,NE,705.0
2,2019-06-03,-1.6,21:13:09,10.0,NNW,1494.0,21:15:47,20.0,NE,1013.0,21:17:40,13.0,ENE,1297.0
3,2019-06-03,-1.6,22:49:23,10.0,WNW,1494.0,22:50:28,17.0,WNW,1107.0,22:50:29,17.0,WNW,1101.0
4,2019-06-04,-3.6,21:59:44,10.0,NW,1494.0,22:02:52,69.0,W,445.0,22:02:55,70.0,WSW,441.0
5,2019-06-05,-3.1,21:10:20,10.0,NW,1493.0,21:13:38,53.0,NE,510.0,21:15:24,23.0,ESE,913.0
6,2019-06-06,-1.8,21:58:01,10.0,WNW,1492.0,22:00:39,21.0,SW,984.0,22:00:47,21.0,SW,987.0
7,2019-06-07,-2.7,21:08:06,10.0,WNW,1491.0,21:11:17,42.0,SW,601.0,21:13:24,18.0,SSE,1086.0
8,2019-06-09,-1.0,21:07:03,10.0,W,1488.0,21:08:45,13.0,SW,1299.0,21:10:27,9.0,SSW,1486.0


In [21]:
def passes(dt0, dt1, topos, satellite, earth, sun):    
    t, y = almanac.find_discrete(ts.utc(dt0), ts.utc(dt1), satellite_pass(manhattan_beach_ca_usa, iss, earth, sun))
    passes = []
    difference = satellite - topos
    for pass_times in grouper(t, 2):
        start = pass_times[0]
        end = pass_times[1]
        t_p = ts.utc(pass_datetimes(start, end))
        y_p = satellite_alt(manhattan_beach_ca_usa, iss, t_p)
        culmination_alt = np.amax(y_p)
        culmination = t_p[np.where(y_p == np.amax(y_p))[0][0]]
        topocentric = difference.at(start)
        alt_start, az_start, d_start = topocentric.altaz('standard')
        topocentric = difference.at(culmination)
        alt_culm, az_culm, d_culm = topocentric.altaz('standard')
        topocentric = difference.at(end)
        alt_end, az_end, d_end = topocentric.altaz('standard')
        if nautical_twilight(topos, earth, sun, start) and nautical_twilight(topos, earth, sun, end):
            umbral_eclipse_rise, _, _ = eclipse_condition(satellite, earth, sun, start)
            umbral_eclipse_set, _, _ = eclipse_condition(satellite, earth, sun, end)
            if umbral_eclipse_rise and umbral_eclipse_set:
                pass_type = 'eclipsed'
            else:
                pass_type = 'visible'
        else:
            pass_type = 'daytime'
        passes.append({ "date": start.astimezone(timezone('US/Pacific')).isoformat(' ', timespec='seconds')[:11],
                          "brightness": np.round(apparent_magnitude(std_intrinsic_mag, 
                                                                d_culm.km, 
                                                                phase_angle(iss, earth, sun, culmination)), 
                                             1),
                          "start": start.astimezone(timezone('US/Pacific')).isoformat(' ', timespec='seconds')[11:19], 
                          "start_alt": np.floor(alt_start.degrees), 
                          "start_az": direction(az_start.degrees),
                          "start_d": np.floor(d_start.km),
                          "culmination": culmination.astimezone(timezone('US/Pacific')).isoformat(' ', 
                                                                                              timespec='seconds')[11:19], 
                          "culm_alt": np.floor(alt_culm.degrees), 
                          "culm_az": direction(az_culm.degrees),
                          "culm_d": np.floor(d_culm.km),
                          "end": end.astimezone(timezone('US/Pacific')).isoformat(' ', timespec='seconds')[11:19], 
                          "end_alt": np.floor(alt_end.degrees), 
                          "end_az": direction(az_end.degrees),
                          "end_d": np.floor(d_end.km),
                          "pass_type": pass_type
                         })
    return pd.DataFrame(passes)[["date", "brightness", "start", "start_alt", "start_az", "start_d",
                                     "culmination", "culm_alt", "culm_az", "culm_d",
                                     "end", "end_alt", "end_az", "end_d", "pass_type"]]

In [22]:
df = passes(dt0, dt1, manhattan_beach_ca_usa, iss, earth, sun)

In [23]:
df

,date,brightness,start,start_alt,start_az,start_d,culmination,culm_alt,culm_az,culm_d,end,end_alt,end_az,end_d,pass_type
0,2019-06-01,1.4,14:43:54,10.0,S,1467.0,14:46:45,26.0,SE,822.0,14:49:36,9.0,ENE,1478.0,daytime
1,2019-06-01,0.6,16:20:38,10.0,W,1475.0,16:23:23,23.0,NW,909.0,16:26:08,9.0,NNE,1485.0,daytime
2,2019-06-01,-3.7,22:51:29,10.0,NW,1496.0,22:54:50,66.0,NE,454.0,22:58:10,9.0,SE,1493.0,visible
3,2019-06-02,3.3,13:55:39,10.0,SSE,1467.0,13:57:27,13.0,SE,1254.0,13:59:15,9.0,E,1474.0,daytime
4,2019-06-02,0.4,15:30:40,10.0,WSW,1472.0,15:33:50,41.0,NW,605.0,15:37:00,9.0,NNE,1483.0,daytime
5,2019-06-02,-2.6,22:02:13,10.0,NNW,1495.0,22:05:20,33.0,NE,705.0,22:08:26,9.0,ESE,1493.0,visible
6,2019-06-02,-1.8,23:39:27,10.0,W,1494.0,23:41:47,17.0,SW,1111.0,23:44:07,9.0,S,1491.0,eclipsed
7,2019-06-03,0.4,14:41:02,10.0,SW,1469.0,14:44:21,84.0,NW,413.0,14:47:40,9.0,NE,1481.0,daytime
8,2019-06-03,1.7,16:20:21,10.0,NW,1481.0,16:21:27,11.0,NNW,1408.0,16:22:33,9.0,N,1484.0,daytime
9,2019-06-03,-1.6,21:13:09,10.0,NNW,1494.0,21:15:45,20.0,NNE,1013.0,21:18:20,9.0,E,1494.0,visible


In [24]:
df[df['pass_type'] == 'visible'][["date", "brightness", "start", "start_alt", "start_az", "start_d",
                                     "culmination", "culm_alt", "culm_az", "culm_d",
                                     "end", "end_alt", "end_az", "end_d"]]

,date,brightness,start,start_alt,start_az,start_d,culmination,culm_alt,culm_az,culm_d,end,end_alt,end_az,end_d
2,2019-06-01,-3.7,22:51:29,10.0,NW,1496.0,22:54:50,66.0,NE,454.0,22:58:10,9.0,SE,1493.0
5,2019-06-02,-2.6,22:02:13,10.0,NNW,1495.0,22:05:20,33.0,NE,705.0,22:08:26,9.0,ESE,1493.0
9,2019-06-03,-1.6,21:13:09,10.0,NNW,1494.0,21:15:45,20.0,NNE,1013.0,21:18:20,9.0,E,1494.0
10,2019-06-03,-2.7,22:49:23,10.0,WNW,1494.0,22:52:28,34.0,SW,701.0,22:55:33,9.0,SSE,1490.0
14,2019-06-04,-3.6,21:59:44,10.0,NW,1494.0,22:03:05,73.0,SW,435.0,22:06:25,9.0,SE,1490.0
17,2019-06-05,-3.1,21:10:20,10.0,NW,1493.0,21:13:37,53.0,NE,510.0,21:16:55,9.0,ESE,1491.0
22,2019-06-06,-1.8,21:58:01,10.0,WNW,1492.0,22:00:38,21.0,SW,984.0,22:03:15,9.0,S,1488.0
26,2019-06-07,-2.7,21:08:06,10.0,WNW,1491.0,21:11:17,42.0,SW,601.0,21:14:28,9.0,SSE,1488.0
34,2019-06-09,-1.0,21:07:03,10.0,W,1488.0,21:08:45,13.0,SW,1299.0,21:10:27,9.0,SSW,1486.0


In [25]:
above_alt_50 = df['culm_alt'] > 50. 
visible = df['pass_type'] == 'visible'
df[visible & above_alt_50]

,date,brightness,start,start_alt,start_az,start_d,culmination,culm_alt,culm_az,culm_d,end,end_alt,end_az,end_d,pass_type
2,2019-06-01,-3.7,22:51:29,10.0,NW,1496.0,22:54:50,66.0,NE,454.0,22:58:10,9.0,SE,1493.0,visible
14,2019-06-04,-3.6,21:59:44,10.0,NW,1494.0,22:03:05,73.0,SW,435.0,22:06:25,9.0,SE,1490.0,visible
17,2019-06-05,-3.1,21:10:20,10.0,NW,1493.0,21:13:37,53.0,NE,510.0,21:16:55,9.0,ESE,1491.0,visible
